Predicting Newsletter Subscription Among Minecraft Server Players Using Gameplay Activity

Background: Minecraft is a popular sandbox video game that allows players to explore, build, and interact within a block-based virtual world. Due to its capability to attract a wide range of players, researchers are using it as a platform for studying behavior.

In this project, a research team at UBC has deployed a custom Minecraft server to collect detailed data on player activity.The server logs their sessions. This behavioral dataset uses variables such as player experience, playtime, and age to specifically explores whether players choose to subscribe to a newsletter related to the game based on these conditions. By analyzing these patterns, we aim to build a predictive model that can help identify the most engaged users based on their in-game behavior. Understanding which player characteristics are predictive of subscription can help the research team better target communications and tailor the gaming experience.

Question: Can player characteristics (such as experience, playtime, age, and gender) predict whether a player subscribes to the game-related newsletter in the Minecraft server dataset?

Data Description:

In [2]:
library(tidyverse)
library(tidymodels)
library(tidyclust)
library(forcats)
library(repr)
options(repr.matrix.max.rows = 6)
source("cleanup.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [3]:
players = read_csv("players.csv")

players

sessions = read_csv("sessions.csv")
sessions

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 249 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): <!DOCTYPE html>

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


<!DOCTYPE html>
<chr>
"<html dir=""ltr"" lang=""en-CA"">"
<head>
"<meta charset=""utf-8"">"
⋮
</div> <!-- #application -->
</body>
</html>


The players dataset used for this analysis consists of 197 observations and 7 variables. It contains information on players of a Minecraft research server, including demographic details, gameplay statistics, and subscription status.

The key variables are summarized below:

Variable 1 (experience): character — player experience level in minecraft Variable 2 (subscribe): logic — whether or not the player is subscribed to the newsletter (TRUE is subscribed, FALSE is not subscribed) Variable 3 (hashedEmail): character — hashed email of the player to protect privacy Variable 4 (played_hours): numeric (decimal number) — number of hour the player has played Variable 5 (name): character — the name of the player Variable 6 (gender): character — gender of the player Variable 7 (Age): numeric (decimal number) — age of the player in years

The data includes both numeric and categorical variables. The response variable for this analysis is subscribe, which indicates whether a player subscribed to the game-related newsletter.

The data includes both numeric and categorical variables. The response variable for this analysis is subscribe, which indicates whether a player subscribed to the game-related newsletter.

Some considerations about the data:

The dataset may contain missing or inconsistent values, which will be addressed during data cleaning. Certain variables, such as hashedEmail and name, serve as unique identifiers and will not be used as predictors. The data were collected through player activity logs and subscription records from the Minecraft research server. Potential limitations include sample size constraints and possible self-reporting bias in demographic variables like gender and age. This dataset provides a comprehensive overview of player characteristics and behavior relevant to predicting newsletter subscription status.